In [17]:
import cv2
import glob
import numpy as np
import math
import os

In [ ]:
#18gig zip file! may take a while
!wget --no-check-certificate \
    http://images.cocodataset.org/zips/train2017.zip \
    -O ./train2017.zip


In [65]:

import zipfile

local_zip = './train2017.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./train2017')
zip_ref.close()

In [5]:
# this selects the first 1975 images, change the regex if you want more
img_paths = glob.glob('./train2017/00000000[0-9]*.jpg')
training_imgs_num = len(img_paths)
print(f'loaded a total of {training_imgs_num} imgs')

loaded a total of 1975 imgs


In [6]:
print(f'Cleaning and saving {training_imgs_num} imgs to ./cleaned-data')
os.mkdir( './cleaned-data/') if not os.path.exists('./cleaned-data/') else None

for img_path in img_paths:
    img = cv2.imread(img_path)
    y,x,_ = img.shape #(y, x) not (x, y)
    
    margin = abs(y-x)/2
    if x > y:        
        # Image is tall
        img = img[:,int(math.floor(margin)):int(math.floor(x-margin))]
    elif y > x:
        # Image is wide
        img = img[int(math.floor(margin)):int(math.floor(y-margin)),:]

    if (img.shape[0] != img.shape[1]):
        print('Dim mismatch')
        
    img = cv2.resize(img, (128,128))
    cv2.imwrite(os.path.join(os.curdir, 'cleaned-data', img_path.split('\\')[-1]), img)

Cleaning and saving 1975 imgs to ./cleaned-data


In [ ]:
# category_mapping = {}

# numeric_mapping = { 'person' : 1, 'vehicle' : 2, 'outdoor' : 3, 'animal' : 4, 'accessory' : 5, 'sports' : 6, \
# 'kitchen' : 7, 'food' : 8, 'furniture' : 9, 'electronic' : 10, 'appliance' : 11, 'indoor' : 12 }

# for dic in instances_df.categories:
#     category_mapping[dic['id']] = numeric_mapping[dic['supercategory']]

# def getSuperCat(cat):
#     return category_mapping[cat]
    

In [40]:
def multihot_encode(data, num_classes=12, clip=False):
    mhe, _ = np.histogram(data,bins=num_classes,range=(0,num_classes-1))
    if clip:
        return np.clip(mhe,0,1)
    
    mhe = [x/sum(mhe) if sum(mhe) > 0 else x/1.0 for x in mhe]
    return mhe
vec_multihot_encode = np.vectorize(multihot_encode)

In [41]:
import pandas as pd

def clean_caption(cap):
    cap = cap.replace('.', ' ')
    cap = cap + "."
    return cap
    


with open('./annotations_trainval2017/annotations/captions_train2017.json') as annot_file:
    captions_df = pd.read_json(annot_file, typ='series')
    annot_df = pd.DataFrame(data=captions_df['annotations'])
    annot_df = annot_df.astype({'image_id': 'int32'})
    annot_df.sort_values(by=['image_id'], axis=0, inplace=True)
    annot_df['caption'] = annot_df['caption'].apply(clean_caption)
    annot_df = annot_df.groupby('image_id')['image_id', 'caption'].agg('|'.join)
    

    with open('./annotations_trainval2017/annotations/instances_train2017.json') as instance_file:
        instances_df = pd.read_json(instance_file, typ='series')
        
        category_mapping = {}

        numeric_mapping = { 'person' : 1, 'vehicle' : 2, 'outdoor' : 3, 'animal' : 4, 'accessory' : 5, 'sports' : 6, \
        'kitchen' : 7, 'food' : 8, 'furniture' : 9, 'electronic' : 10, 'appliance' : 11, 'indoor' : 12 }

        for dic in instances_df.categories:
            category_mapping[dic['id']] = numeric_mapping[dic['supercategory']]
        
        image_cats = {i : [] for i in list(annot_df.index)}
        image_supercats = {i : [] for i in list(annot_df.index)}
        
        for row in instances_df.annotations:
            image_cats[row['image_id']].append(row['category_id'])
            image_supercats[row['image_id']].append(category_mapping[row['category_id']])
        
        annot_df.insert(1, 'categories', image_cats.values())
        annot_df.insert(2, 'super_categories', list(map(multihot_encode, list(image_supercats.values()))))

In [9]:
'asfd' + "."

'asfd.'

In [42]:
annot_df

,caption,categories,super_categories
image_id,,,
9,Closeup of bins of food that include broccoli ...,"[51, 51, 56, 51, 55, 55, 55, 55]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0.6..."
25,A giraffe standing up nearby a tree .|A giraff...,"[25, 25]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
30,White vase with different colored flowers sitt...,"[64, 86]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
34,Zebra reaching its head down to ground where g...,[24],"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
36,A woman with an umbrella near the sea.|A girl ...,"[28, 1]","[0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, ..."
42,a shoe rack with some shoes and a dog sleeping...,[18],"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
49,A pair of horses performing tricks in a field ...,"[19, 19, 1, 1, 1, 64, 1, 1, 1]","[0.0, 0.6666666666666666, 0.0, 0.0, 0.22222222..."
61,They are brave for riding in the jungle on tho...,"[1, 1, 22, 22, 1]","[0.0, 0.6, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, ..."
64,A clock is seen in front of a tall tree .|A ve...,"[3, 8, 13, 85]","[0.0, 0.0, 0.6666666666666666, 0.3333333333333..."


In [26]:
instances_df.categories

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [43]:
annot_df.to_csv('./coco-captions-with-categories.csv')